# Convolutional Neural Network

### Importing the libraries

In [1]:
#!pip install --upgrade tensorflow keras Pillow

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2022-03-12 12:22:14.999827: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-12 12:22:14.999842: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

2022-03-12 12:22:17.960022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-12 12:22:17.960284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-12 12:22:17.960329: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-12 12:22:17.960365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-12 12:22:17.960398: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.1))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.1))

### Adding a third convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.1))

### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation='tanh'))

### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)

Epoch 1/30
250/250 [==============================] - 14s 54ms/step - loss: 0.6783 - accuracy: 0.6009 - val_loss: 0.5774 - val_accuracy: 0.7110
Epoch 2/30
250/250 [==============================] - 13s 53ms/step - loss: 0.5926 - accuracy: 0.6890 - val_loss: 0.5771 - val_accuracy: 0.6970
Epoch 3/30
250/250 [==============================] - 13s 53ms/step - loss: 0.5546 - accuracy: 0.7147 - val_loss: 0.5367 - val_accuracy: 0.7415
Epoch 4/30
250/250 [==============================] - 13s 54ms/step - loss: 0.5426 - accuracy: 0.7244 - val_loss: 0.5265 - val_accuracy: 0.7445
Epoch 5/30
250/250 [==============================] - 13s 53ms/step - loss: 0.5282 - accuracy: 0.7345 - val_loss: 0.5185 - val_accuracy: 0.7455
Epoch 6/30
250/250 [==============================] - 13s 53ms/step - loss: 0.5187 - accuracy: 0.7405 - val_loss: 0.5086 - val_accuracy: 0.7605
Epoch 7/30
250/250 [==============================] - 14s 55ms/step - loss: 0.5102 - accuracy: 0.7460 - val_loss: 0.4877 - val_accuracy:

## Part 4 - Making a single prediction

In [16]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] > 0.8:
  prediction = 'dog'
else:
  prediction = 'cat'

In [17]:
print(prediction)

dog


## Fim